In [2]:
import numpy as np
from sklearn import metrics
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, datasets
import torchvision.transforms as transforms
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [3]:
print(f"Is GPU available? {torch.cuda.is_available()}")
print(f"Number of available devices: {torch.cuda.device_count()}")
print(f"Index of current device: {torch.cuda.current_device()}")
print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

Is GPU available? True
Number of available devices: 1
Index of current device: 0
Device name: NVIDIA GeForce GTX 1650 with Max-Q Design


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


#defaults
BATCH_SIZE = 128
N_EPOCHS = 10
LEARNING_RATE = 0.001

# Nacitanie dat

In [5]:
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

class notMNIST(Dataset):
    def __init__(self, data, labels):
        self.images = data
        self.labels = labels

        self.transformation = transforms.Compose([
                                                  
            transforms.ToTensor(),
            transforms.ConvertImageDtype(dtype=torch.float32),
        ])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]
        img_tensor = self.transformation(img)
        y_tensor = torch.tensor(label, dtype=torch.long)
        return img_tensor.view(1, 28, 28), y_tensor


data = loadmat('notMNIST_small.mat')
print(data.keys())

images = data['images']
labels = data['labels']

print(images.shape)
images = [images[:, :, i] for i in range(0, images.shape[2])]
images = np.asarray(images)
print(images.shape)
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.4, shuffle=True)

train_dataset = notMNIST(x_train, y_train)
test_dataset = notMNIST(x_test, y_test)

trainloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

dict_keys(['__header__', '__version__', '__globals__', 'images', 'labels'])
(28, 28, 18724)
(18724, 28, 28)


# Import sieti

In [2]:
!jupyter nbconvert --to python notMNIST_networks.ipynb

#The classes cannot be run from a jupyter notebook, this executes the script and converts it to a python script

from notMNIST_networks import MLP, CustomCNN

[NbConvertApp] Converting notebook notMNIST_networks.ipynb to python
[NbConvertApp] Writing 1921 bytes to notMNIST_networks.py


# Trenovacia funkcia


In [9]:
def training_function(model, trainloader, writer, batch_size, n_epochs, learning_rate):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model = model.to(device)

    model.train()

    acc_history = []
    loss_history = []
    final_labels = []
    final_predicted = []

    for epoch in trange(1, n_epochs + 1, desc="1st loop"):
        epoch_loss = 0
        n_batches = len(train_dataset) // batch_size
        correct = 0
        total = 0
        accuracy_train = 0

        for step, (images, labels) in enumerate(tqdm(trainloader, desc="Epoch {}/{}".format(epoch, N_EPOCHS))):

            images = images.to(device)
            labels = labels.to(device)
            
            # Dopredne sirenie, 
            # ziskame pravdepodobnosti tried tym, ze posleme do modelu vstupy
            outputs = model(images)

            # Vypocitame chybu algoritmu       
            loss = criterion(outputs, labels)
            
            # Uspesnost algoritmu
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy_train = correct / total
            epoch_loss += loss.item() 
            
            # Je vhodne zavolat zero_grad() pred zavolanim spatneho sirenia 
            # pre vynulovanie gradientov z predosleho volania loss.backward()
            optimizer.zero_grad()

            # Spatne sirenie chyby, vypocitaju sa gradienty
            loss.backward()
            
            # Aktualizacia vah pomocou optimalizatora
            optimizer.step()

            if step % n_batches == 0 and step != 0:
                epoch_loss = epoch_loss / n_batches

                writer.add_scalar(
                    'training loss',
                    epoch_loss,
                    epoch
                )
                
                writer.add_scalar(
                    'training accuracy',
                    accuracy_train,
                    epoch
                )

                acc_history.append(accuracy_train)
                loss_history.append(epoch_loss)
                print("Epoch {}, Loss {:.6f}, Accuracy {:.2f}% ".format(epoch, epoch_loss, accuracy_train * 100))
                epoch_loss = 0

                #to display knovolution kernels/filters?
                #print(model.layer1[0].conv1.weight[0][0])
                #print(model.layer2[0].conv1.weight[0][0])
                #print(model.layer3[0].conv1.weight[0][0])

            final_predicted += predicted.tolist()
            final_labels += labels.tolist()
            torch.cuda.empty_cache()

            #writing training data
            
    writer.add_hparams(
        {
        'optimizer': optimizer.__class__.__name__,
        'lr': LEARNING_RATE, 
        'batch_size': BATCH_SIZE
        },
        {
        'hparam/train/accuracy': accuracy_train,
        }
    )
    writer.close()

# Trenovanie

In [10]:


model_cnn_1 = CustomCNN()

model_mlp = MLP()
writer_mlp = SummaryWriter('./runs/experiment_MLP')

training_function(model_mlp,trainloader,writer_mlp,BATCH_SIZE,N_EPOCHS,LEARNING_RATE)




# model = CustomCNN()


1st loop:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10:   0%|          | 0/88 [00:00<?, ?it/s]

C:\Users\micha\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 1, Loss 2.308039, Accuracy 17.89% 


Epoch 2/10:   0%|          | 0/88 [00:00<?, ?it/s]

...

Epoch 8, Loss 2.324545, Accuracy 16.29% 


Epoch 9/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 9, Loss 2.328783, Accuracy 15.89% 


Epoch 10/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 10, Loss 2.311205, Accuracy 17.62% 


# Ukladanie modelu

In [13]:
PATH_MLP = "./model_mlp.pt"
PATH_CNN_1= "./model_cnn_1.pt"
PATH_CNN_2= "./model_cnn_2.pt"
PATH_CNN_resnet= "./model_cnn_resnet.pt"

#saving MLP
torch.save(model_mlp.state_dict(), PATH_MLP)

#torch.save(model.state_dict(), PATH)